In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
from scipy import stats

# News

In [ ]:
Snopes_harvey=pd.read_excel('/....../News/LIWC2015 Results (snopos_harvey_cleaned).xlsx')
Snopes_irma=pd.read_excel('......./News/LIWC2015 Results (snopos_irma_cleaned).xlsx')
CNN_harvey=pd.read_excel('......./News/LIWC2015 Results (CNN_harvey_cleaned).xlsx')
CNN_irma=pd.read_excel('......./News/LIWC2015 Results (CNN_irma_cleaned).xlsx')

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

def per_unit(Mental_health,minn,maxx): 
    scaler = MinMaxScaler(feature_range=(minn, maxx))
    Mental_health=np.array(Mental_health)
    scaler.fit(Mental_health.reshape(-1, 1))
    Mental_health = scaler.transform(Mental_health.reshape(-1, 1))
    return Mental_health
CNN_irma_pos=(CNN_irma['posemo']-CNN_irma['negemo'])/CNN_irma['WC']
min_val=0.5-(((0-CNN_irma_pos.min())*0.5)/CNN_irma_pos.max())
CNN_irma_pos=per_unit(CNN_irma_pos,min_val,0.9999)        ##needed

CNN_harvey_pos=(CNN_harvey['posemo']-CNN_harvey['negemo'])/CNN_harvey['WC']
max_val=((CNN_harvey_pos.max()*0.5)/(0-CNN_harvey_pos.min()))+0.5
CNN_harvey_pos=per_unit(CNN_harvey_pos,0.0001,max_val)     ##needed
fake_harvey= per_unit(Snopes_harvey['frequency'],0,1)     ##needed
fake_irma= per_unit(Snopes_irma['frequency'],0,1)         ##needed 

# Emergency services

In [ ]:
EMS_harvey=pd.read_excel('......./Emergency services/Emergency services-harvey.xlsx')
EMS_irma=pd.read_excel('......./Emergency services/Emergency services-irma.xlsx')
EMS_harvey=EMS_harvey.drop(['end'],axis=1)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

########### Missing Values ##############
list_harvey=EMS_harvey.columns
list_irma=EMS_irma.columns
for i in  list_harvey[1:]:
   EMS_harvey[i]=EMS_harvey[i].interpolate().astype('int32')

EMS_harvey=EMS_harvey[1:]
for i in  list_irma[1:]:
   EMS_irma[i]=EMS_irma[i].interpolate().astype('int32')

###########per unit   
def per_unit(Mental_health): 
    scaler = MinMaxScaler(feature_range=(0, 1))
    Mental_health=np.array(Mental_health)
    scaler.fit(Mental_health.reshape(-1, 1))
    Mental_health = scaler.transform(Mental_health.reshape(-1, 1))
    return Mental_health

for i in  list_harvey[1:]:
   EMS_harvey[i]=per_unit(EMS_harvey[i])

for i in  list_irma[1:]:
   EMS_irma[i]=per_unit(EMS_irma[i])
  
EMS_harvey_mean=EMS_harvey.drop(['Time'],axis=1)
EMS_harvey_mean=EMS_harvey_mean.mean(axis=1)  ##needed
EMS_irma_mean=EMS_irma.drop(['Time'],axis=1)
EMS_irma_mean=EMS_irma_mean.mean(axis=1)       ##needed



#Data


**Electricity data:**

https://poweroutage.us/

TADS data  NERC: TADS-like public data for transamission:

https://transmission.bpa.gov/Business/Operations/Outages/

https://www.nerc.com/pa/RAPA/tads/Pages/default.aspx

https://www.eia.gov/tools/a-z/index.php?id=I


**Hurrican Irma**

https://data.floridatoday.com/storm-power-outages/

Power outage:  https://twitter.com/PowerOutage_us/status/911192590636240896

https://www.eia.gov/special/disruptions/archive/hurricane/irma/




**Hurricane Harvey:**

http://www.ercot.com/help/harvey

https://www.eia.gov/todayinenergy/detail.php?id=32892

In [ ]:
import os, sys
import pandas as pd
import numpy as np
import math
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
from scipy import stats
from scipy.optimize import curve_fit

In [ ]:
#Irma
Irma_dt_adult = pd.read_excel('......./irma/Results (Irma_Hourly).xlsx')
Irma_dt_adult_3 = pd.read_excel('......./irma/Results (Irma_3hour).xlsx')
Irma_dt_adult_d = pd.read_excel('......./irma/Results (Irma_Daily).xlsx')
Irma_dt_adult_d_130 = pd.read_excel('......./Initial tweets/LIWC2015 Results (irma250_Daily).xlsx')

# Harvey
Harvey_dt_adult = pd.read_excel('......./Harvey/Results (Harvey_Hourly).xlsx')
Harvey_dt_adult_3 = pd.read_excel('......./Harvey/Results (Harvey_3hour).xlsx')
Harvey_dt_adult_d = pd.read_excel('......./Harvey/Results (Harvey_Daily).xlsx')
Harvey_dt_adult_d_130 = pd.read_excel('......./LIWC2015 Results (Harvey250_dayly).xlsx')


Irma_severity=[1,2,3,4,5,5,5,4,4,4,3,2,1]             #start 09-01
Harvey_sevirty=[4,4,4,4,4,4,3,2,1,1,0,0,0,0,0,0,0,0]  #start 08-25
List=[Irma_dt_adult,Irma_dt_adult_3,Harvey_dt_adult,Harvey_dt_adult_3]


def filling_missing_value(X):
  for i in range(len(X)):
    Y=''
    Z='' 
    for j  in range(len(X)):
        if X[i-j]!='': Y= X[i-j]
        if Y !='': break    
    for k in range(len(X)):
        if X[i+k]!='': Z= X[i+k]
        if Z !='': break 
    if  j+k !=0: X[i]= (k/(j+k))*Y + (j/(j+k))* Z; 
  return X


for i in range(len(List)):
    List[i]['severity']=''
    List[i]['time_lable']=''
    List[i]['Event']=''
for i in range(len(List)):
    for j in range(List[i].shape[0]):
       X=str(int(List[i]['time'][j])-2017000000)  
       List[i].at[j,'time_lable']=X[:-4]+'-'+X[-4:-2]+'-'+X[-2:]
    if i<= 1:
       List[i].at[0,'severity']=Irma_severity[0]/5
       List[i].at[List[i].shape[0]-1,'severity']=Irma_severity[len(Irma_severity)-1]/5
       K=int(List[i].shape[0]/len(Irma_severity))
       n=K
       for j in range(len(Irma_severity)-2):
           List[i].at[n,'severity']=Irma_severity[j+1]/5
           n=n+K
       List[i]['severity']=filling_missing_value(List[i]['severity'])
       for j in range(List[i].shape[0]):
           List[i].at[j,'Event']='Irma 2017, Florida'
    if i>1:
        List[i].at[0,'severity']=Harvey_sevirty[0]/5
        List[i].at[List[i].shape[0]-1,'severity']=Harvey_sevirty[len(Harvey_sevirty)-1]/5
        K=int(List[i].shape[0]/len(Harvey_sevirty))
        n=K
        for j in range(len(Harvey_sevirty)-2):
           List[i].at[n,'severity']=Harvey_sevirty[j+1]/5
           n=n+K
        List[i]['severity']=filling_missing_value(List[i]['severity']) 
        for j in range(List[i].shape[0]):
           List[i].at[j,'Event']='Harvey 2017, Texas'

In [ ]:
###for  News and emergency services
#List=[Irma_dt_adult,Irma_dt_adult_3,Harvey_dt_adult,Harvey_dt_adult_3]
List=[List[0],List[1],List[2],List[3]]

for i in range(len(List)):
    List[i]['fake']=''
    List[i]['CNN_pos']=''
    List[i]['EMS']=''
    List[i]['info_seek_GT']=''

def  fillin_mising_hour(Irma_severity,List,feature):
    #List[feature]=''
    List.at[0,feature]=Irma_severity[0]
    List.at[List.shape[0]-1,feature]=Irma_severity[len(Irma_severity)-1]
    K=int(List.shape[0]/len(Irma_severity))
    n=K
    for j in range(len(Irma_severity)-2):
           List.at[n,feature]=Irma_severity[j+1]
           n=n+K
    List[feature]=filling_missing_value(List[feature])
    return  List


for i in range(len(List)):
    if i<= 1:
                  List[i]=fillin_mising_hour(fake_irma,List[i],'fake')
                  List[i]=fillin_mising_hour(CNN_irma_pos,List[i],'CNN_pos')
                  List[i]=fillin_mising_hour(EMS_irma_mean,List[i],'EMS')
                  List[i]=fillin_mising_hour(info_irma,List[i],'info_seek_GT')
    if i>1:
                  List[i]=fillin_mising_hour(fake_harvey,List[i],'fake')
                  List[i]=fillin_mising_hour(CNN_harvey_pos,List[i],'CNN_pos')
                  List[i]=fillin_mising_hour(np.array(EMS_harvey_mean),List[i],'EMS')
                  List[i]=fillin_mising_hour(info_harvey,List[i],'info_seek_GT')
